In [1]:
import pywt
import os
from pylab import *
from numpy import*
import pandas as pd

In [244]:
df = pd.read_csv('train/train_meta.csv')

In [3]:
df.head()

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,record_name,group
0,9,18792.0,55.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-12-08 09:44:43,...,", I-AVR,",NaN,NaN,NaN,NaN,10,records100/00000/00009_lr,records500/00000/00009_hr,00009_hr,1
1,34,13619.0,56.0,0,NaN,NaN,2.0,0.0,CS-12 E,1985-02-10 12:04:03,...,", alles,",NaN,NaN,NaN,NaN,9,records100/00000/00034_lr,records500/00000/00034_hr,00034_hr,3
2,43,11315.0,25.0,1,NaN,63.0,2.0,0.0,CS-12 E,1985-03-14 09:38:53,...,NaN,NaN,NaN,NaN,NaN,8,records100/00000/00043_lr,records500/00000/00043_hr,00043_hr,3
3,52,18153.0,35.0,0,NaN,82.0,2.0,0.0,CS-12 E,1985-05-18 08:15:54,...,NaN,NaN,NaN,NaN,NaN,1,records100/00000/00052_lr,records500/00000/00052_hr,00052_hr,2
4,57,16063.0,26.0,0,NaN,93.0,2.0,0.0,CS-12 E,1985-06-06 11:32:43,...,NaN,NaN,NaN,NaN,NaN,10,records100/00000/00057_lr,records500/00000/00057_hr,00057_hr,3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ecg_id                        2101 non-null   int64  
 1   patient_id                    2101 non-null   float64
 2   age                           2101 non-null   float64
 3   sex                           2101 non-null   int64  
 4   height                        689 non-null    float64
 5   weight                        905 non-null    float64
 6   nurse                         1947 non-null   float64
 7   site                          2100 non-null   float64
 8   device                        2101 non-null   object 
 9   recording_date                2101 non-null   object 
 10  report                        2101 non-null   object 
 11  scp_codes                     2101 non-null   object 
 12  heart_axis                    1231 non-null   object 
 13  inf

In [245]:
gts = pd.read_csv('train/train_gts.csv')

In [246]:
myocard_list = list(gts['myocard'])
df.insert(loc=len(df.columns), column='myocard', value=myocard_list)

In [247]:
d1 = list(df['static_noise'])
d2 = list(df['burst_noise'])
d3 = list(df['electrodes_problems'])
d4 = list(df['baseline_drift'])

In [248]:
df = df[(df.static_noise != d1) & (df.burst_noise != d2) & (df.electrodes_problems != d3) & (df.baseline_drift != d4)]
#удаление строк, содержащих шум, проблемы с электродами, смещение базовой линии/скачок

In [249]:
df[['weight', 'height']] = new_df[['weight', 'height']]. fillna(new_df[['weight', 'height']].mean())

combo_info = df['weight'].astype(str) + '_' + df['height'].astype(str) + '_' + df['age'].astype(str) + '_' + df['sex'].astype(str)
df.insert(0, 'combo_info', combo_info)
df['combo_info'] = df.groupby('combo_info')['myocard'].transform('mean')
#объединение данных веса, роста, возраста и пола(во избежание переобучения)

In [250]:
df['scp_codes'] = df.groupby('scp_codes')['myocard'].transform('mean') #стандарт для обмена цифровыми ЭКГ

df['infarction_stadium1'] = df.groupby('infarction_stadium1')['myocard'].transform('mean') #стадия инфаркта
df['infarction_stadium2'] = df.groupby('infarction_stadium2')['myocard'].transform('mean') #вторая стадия инфаркта
df[['infarction_stadium1', 'infarction_stadium2']] = df[['infarction_stadium1', 'infarction_stadium2']].fillna(0)
#заполнение пустых значений нулём

df['heart_axis'] = df.groupby('heart_axis')['myocard'].transform('mean') #электрическая ось сердца
oft_value1 = df['heart_axis'].value_counts().idxmax() 
df['heart_axis'] = df['heart_axis'].fillna(oft_value1) #заполнение NaN наиболее встречающимся значением

df['device'] = df.groupby('device')['myocard'].transform('mean') #устройство записи
df['report'] = df.groupby('report')['myocard'].transform('mean') #краткий доклад о экг пациента

df['extra_beats'] = df.groupby('extra_beats')['myocard'].transform('mean')
#дополнительные удары сердца(обычно никак не влияют на здоров/болен)
df['extra_beats'] = df['extra_beats'].fillna(0)

oft_value2 = df['nurse'].value_counts().idxmax() #"номер" медсестры
df['nurse'] = df['nurse'].fillna(oft_value2)

oft_value3 = df['validated_by'].value_counts().idxmax() #"номер" утверждающего кардиолога
df['validated_by'] = df['validated_by'].fillna(oft_value3)

df['pacemaker'] = df.groupby('pacemaker')['myocard'].transform('mean')
#кардиостимулятор(регулирует функцию электрической проводящей системы сердца)
df['pacemaker'] = df['pacemaker'].fillna(0)

oft_value4 = df['site'].value_counts().idxmax() #"номер" сайта записи
df['site'] = df['site'].fillna(oft_value4)

In [251]:
df.drop(['recording_date', 'filename_lr', 'filename_hr', 'record_name',
         'age', 'sex', 'height', 'weight', 'static_noise', 
         'burst_noise', 'electrodes_problems', 'baseline_drift'], axis= 1 , inplace= True )

In [252]:
df.head()

,combo_info,ecg_id,patient_id,nurse,site,device,report,scp_codes,heart_axis,infarction_stadium1,infarction_stadium2,validated_by,second_opinion,initial_autogenerated_report,validated_by_human,extra_beats,pacemaker,strat_fold,group,myocard
2,0.0,43,11315.0,2.0,0.0,0.229167,0.3,0.219355,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,8,3,0
3,0.0,52,18153.0,2.0,0.0,0.229167,0.0,0.190476,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,1,2,0
4,0.0,57,16063.0,2.0,0.0,0.229167,0.3,0.219355,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,10,3,0
5,0.0,61,14660.0,2.0,0.0,0.229167,0.3,0.219355,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,6,1,0
8,0.0,108,11810.0,0.0,0.0,0.229167,0.0,0.352941,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,8,3,0


In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1633 entries, 2 to 2100
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   combo_info                    1633 non-null   float64
 1   ecg_id                        1633 non-null   int64  
 2   patient_id                    1633 non-null   float64
 3   nurse                         1633 non-null   float64
 4   site                          1633 non-null   float64
 5   device                        1633 non-null   float64
 6   report                        1633 non-null   float64
 7   scp_codes                     1633 non-null   float64
 8   heart_axis                    1633 non-null   float64
 9   infarction_stadium1           1633 non-null   float64
 10  infarction_stadium2           1633 non-null   float64
 11  validated_by                  1633 non-null   float64
 12  second_opinion                1633 non-null   bool   
 13  ini

In [254]:
values = df.values
X = values[:,0:18]
y = values[:,19]
y=y.astype('int')

In [255]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.35, random_state=0)

In [256]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [257]:
cat = CatBoostClassifier(depth=8,  learning_rate=0.05)
cat.fit(X_train, y_train)

0:	learn: 0.6167517	total: 6.05ms	remaining: 6.04s
1:	learn: 0.5428700	total: 10.6ms	remaining: 5.27s
2:	learn: 0.4880502	total: 14.5ms	remaining: 4.83s
3:	learn: 0.4381187	total: 19.5ms	remaining: 4.87s
4:	learn: 0.4026607	total: 23.5ms	remaining: 4.67s
5:	learn: 0.3563733	total: 27.9ms	remaining: 4.62s
6:	learn: 0.3244671	total: 33.1ms	remaining: 4.69s
7:	learn: 0.2905341	total: 37.6ms	remaining: 4.67s
8:	learn: 0.2641386	total: 41.8ms	remaining: 4.6s
9:	learn: 0.2465584	total: 46.6ms	remaining: 4.61s
10:	learn: 0.2277389	total: 50.7ms	remaining: 4.56s
11:	learn: 0.2094534	total: 55.2ms	remaining: 4.54s
12:	learn: 0.1961224	total: 59.5ms	remaining: 4.52s
13:	learn: 0.1849655	total: 63.6ms	remaining: 4.48s
14:	learn: 0.1739125	total: 67.4ms	remaining: 4.42s
15:	learn: 0.1644354	total: 72.6ms	remaining: 4.46s
16:	learn: 0.1592868	total: 78ms	remaining: 4.51s
17:	learn: 0.1528771	total: 81.9ms	remaining: 4.47s
18:	learn: 0.1444440	total: 87.4ms	remaining: 4.51s
19:	learn: 0.1374272	tota

193:	learn: 0.0124451	total: 801ms	remaining: 3.33s
194:	learn: 0.0123598	total: 806ms	remaining: 3.33s
195:	learn: 0.0122403	total: 810ms	remaining: 3.32s
196:	learn: 0.0121311	total: 815ms	remaining: 3.32s
197:	learn: 0.0120374	total: 819ms	remaining: 3.32s
198:	learn: 0.0119420	total: 822ms	remaining: 3.31s
199:	learn: 0.0118638	total: 826ms	remaining: 3.3s
200:	learn: 0.0117780	total: 830ms	remaining: 3.3s
201:	learn: 0.0116960	total: 834ms	remaining: 3.29s
202:	learn: 0.0115997	total: 838ms	remaining: 3.29s
203:	learn: 0.0115182	total: 842ms	remaining: 3.28s
204:	learn: 0.0114430	total: 845ms	remaining: 3.28s
205:	learn: 0.0113379	total: 849ms	remaining: 3.27s
206:	learn: 0.0112265	total: 853ms	remaining: 3.27s
207:	learn: 0.0111531	total: 856ms	remaining: 3.26s
208:	learn: 0.0110544	total: 860ms	remaining: 3.25s
209:	learn: 0.0109475	total: 863ms	remaining: 3.25s
210:	learn: 0.0108389	total: 867ms	remaining: 3.24s
211:	learn: 0.0107824	total: 871ms	remaining: 3.24s
212:	learn: 0.

395:	learn: 0.0038925	total: 1.61s	remaining: 2.45s
396:	learn: 0.0038780	total: 1.61s	remaining: 2.45s
397:	learn: 0.0038673	total: 1.62s	remaining: 2.44s
398:	learn: 0.0038568	total: 1.62s	remaining: 2.44s
399:	learn: 0.0038424	total: 1.63s	remaining: 2.44s
400:	learn: 0.0038290	total: 1.63s	remaining: 2.44s
401:	learn: 0.0038124	total: 1.63s	remaining: 2.43s
402:	learn: 0.0037953	total: 1.64s	remaining: 2.43s
403:	learn: 0.0037831	total: 1.64s	remaining: 2.42s
404:	learn: 0.0037693	total: 1.65s	remaining: 2.42s
405:	learn: 0.0037550	total: 1.65s	remaining: 2.41s
406:	learn: 0.0037470	total: 1.65s	remaining: 2.41s
407:	learn: 0.0037363	total: 1.66s	remaining: 2.4s
408:	learn: 0.0037240	total: 1.66s	remaining: 2.4s
409:	learn: 0.0037161	total: 1.67s	remaining: 2.4s
410:	learn: 0.0037072	total: 1.67s	remaining: 2.39s
411:	learn: 0.0036951	total: 1.67s	remaining: 2.39s
412:	learn: 0.0036821	total: 1.68s	remaining: 2.38s
413:	learn: 0.0036705	total: 1.68s	remaining: 2.38s
414:	learn: 0.0

603:	learn: 0.0022841	total: 2.41s	remaining: 1.58s
604:	learn: 0.0022797	total: 2.42s	remaining: 1.58s
605:	learn: 0.0022760	total: 2.42s	remaining: 1.57s
606:	learn: 0.0022725	total: 2.42s	remaining: 1.57s
607:	learn: 0.0022690	total: 2.43s	remaining: 1.56s
608:	learn: 0.0022656	total: 2.43s	remaining: 1.56s
609:	learn: 0.0022609	total: 2.44s	remaining: 1.56s
610:	learn: 0.0022574	total: 2.44s	remaining: 1.55s
611:	learn: 0.0022504	total: 2.44s	remaining: 1.55s
612:	learn: 0.0022460	total: 2.45s	remaining: 1.54s
613:	learn: 0.0022424	total: 2.45s	remaining: 1.54s
614:	learn: 0.0022389	total: 2.45s	remaining: 1.54s
615:	learn: 0.0022339	total: 2.46s	remaining: 1.53s
616:	learn: 0.0022296	total: 2.46s	remaining: 1.53s
617:	learn: 0.0022253	total: 2.46s	remaining: 1.52s
618:	learn: 0.0022211	total: 2.47s	remaining: 1.52s
619:	learn: 0.0022176	total: 2.47s	remaining: 1.51s
620:	learn: 0.0022126	total: 2.48s	remaining: 1.51s
621:	learn: 0.0022093	total: 2.48s	remaining: 1.51s
622:	learn: 

762:	learn: 0.0017270	total: 3s	remaining: 933ms
763:	learn: 0.0017240	total: 3.01s	remaining: 929ms
764:	learn: 0.0017205	total: 3.01s	remaining: 925ms
765:	learn: 0.0017180	total: 3.02s	remaining: 921ms
766:	learn: 0.0017147	total: 3.02s	remaining: 918ms
767:	learn: 0.0017123	total: 3.02s	remaining: 914ms
768:	learn: 0.0017096	total: 3.03s	remaining: 910ms
769:	learn: 0.0017071	total: 3.03s	remaining: 906ms
770:	learn: 0.0017050	total: 3.04s	remaining: 902ms
771:	learn: 0.0017020	total: 3.04s	remaining: 899ms
772:	learn: 0.0016998	total: 3.05s	remaining: 895ms
773:	learn: 0.0016978	total: 3.05s	remaining: 891ms
774:	learn: 0.0016947	total: 3.05s	remaining: 887ms
775:	learn: 0.0016916	total: 3.06s	remaining: 883ms
776:	learn: 0.0016895	total: 3.06s	remaining: 879ms
777:	learn: 0.0016871	total: 3.06s	remaining: 875ms
778:	learn: 0.0016852	total: 3.07s	remaining: 871ms
779:	learn: 0.0016832	total: 3.07s	remaining: 867ms
780:	learn: 0.0016803	total: 3.08s	remaining: 863ms
781:	learn: 0.0

971:	learn: 0.0012846	total: 3.8s	remaining: 110ms
972:	learn: 0.0012826	total: 3.81s	remaining: 106ms
973:	learn: 0.0012815	total: 3.81s	remaining: 102ms
974:	learn: 0.0012788	total: 3.81s	remaining: 97.8ms
975:	learn: 0.0012776	total: 3.82s	remaining: 93.9ms
976:	learn: 0.0012748	total: 3.82s	remaining: 90ms
977:	learn: 0.0012738	total: 3.83s	remaining: 86.1ms
978:	learn: 0.0012721	total: 3.83s	remaining: 82.2ms
979:	learn: 0.0012710	total: 3.83s	remaining: 78.3ms
980:	learn: 0.0012698	total: 3.84s	remaining: 74.3ms
981:	learn: 0.0012680	total: 3.84s	remaining: 70.4ms
982:	learn: 0.0012661	total: 3.85s	remaining: 66.5ms
983:	learn: 0.0012642	total: 3.85s	remaining: 62.6ms
984:	learn: 0.0012625	total: 3.85s	remaining: 58.7ms
985:	learn: 0.0012610	total: 3.86s	remaining: 54.8ms
986:	learn: 0.0012593	total: 3.86s	remaining: 50.8ms
987:	learn: 0.0012566	total: 3.86s	remaining: 46.9ms
988:	learn: 0.0012549	total: 3.87s	remaining: 43ms
989:	learn: 0.0012533	total: 3.87s	remaining: 39.1ms
9

In [258]:
f1_score(y_train, cat.predict(X_train), average = 'macro')

1.0

In [259]:
f1_score(y_test, cat.predict(X_test), average = 'macro')

0.9395509007598828